In [1]:
import pydicom as py
import os
import tensorflow as tf
import tensorflow_hub as hub
import glob
import numpy as np

In [2]:
dicom_dir = './data/DICOM'

dicom_file_paths = glob.glob('{}/*.dcm'.format(dicom_dir))

In [3]:
key_fields = [
    'Modality', 'PatientAge', 'PatientID', 
    'PatientPosition', 'PatientSex', 
    'PixelData', 'StudyDescription', 'BodyPartExamined'
]

In [4]:
def check_dicom(filepath):
    
    splitted = filepath.split('/')
    
    filename = splitted[len(splitted)-1]
    
    print('Load file {} ...'.format(filename))
    
    try:
        
        dcm_data = py.dcmread(filepath)
    
    except:
        
        print('File {} is not a valid DICOM file'.format(filename))
        
        return None
    
    else:
        
        dcm_keys = dcm_data.dir()

        contains_all_fields =  all(item in dcm_keys for item in key_fields)

        if not contains_all_fields:

            print('File {} does not contain all required fields'.format(filename))

            return None

        info_dict = {field: dcm_data[field].value for field in key_fields}

        if info_dict['Modality'] != 'DX':

            print('The modality of File {} is not Digital Radiography(DX) - {}'.format(filename, info_dict['Modality']))

            return None

        if info_dict['PatientPosition'] != 'AP' and info_dict['PatientPosition'] != 'PA':

            print('The view position of File {} is not valid - {}'.format(filename, info_dict['PatientPosition']))

            return None

        if info_dict['BodyPartExamined'] != 'CHEST':

            print('The body part examined of File {} is not CHEST - {}'.format(filename, info_dict['BodyPartExamined']))

            return None

        return dcm_data.pixel_array
    

In [5]:
for filepath in dicom_file_paths:
    
    np_img = check_dicom(filepath)
    
    if np_img is None:
        continue
        
    print(np_img.shape)

Load file test4.dcm ...
The body part examined of File test4.dcm is not CHEST - RIBCAGE
Load file test5.dcm ...
The modality of File test5.dcm is not Digital Radiography(DX) - CT
Load file test6.dcm ...
The view position of File test6.dcm is not valid - XX
Load file test2.dcm ...
(1024, 1024)
Load file test3.dcm ...
(1024, 1024)
Load file test1.dcm ...
(1024, 1024)
